# Dependency management for programs

In this document, we will learn how to install custom dependencies to your program.

Let's create another file with our new program [./source_files/program_3.py](./source_files/program_3.py). 

For the sake of this example, let's use `qiskit-experiments` as our custom dependency. We will use randomized benchmarking (RB) circuits from qiskit-experiments, composed with the circuit from the input arguments for measurement.

Here's what the file would look like:

```python
# source_files/program_3.py

from quantum_serverless import get_arguments, save_result

from qiskit.primitives import Sampler
from qiskit_experiments.library import StandardRB


arguments = get_arguments()

circuit = arguments.get("circuit")

rb = StandardRB(
    physical_qubits=(1,),
    lengths=list(range(1, 300, 30)),
    seed=42
)
composed = circuit.compose(rb.circuits()[0])

sampler = Sampler()

quasi_dists = sampler.run(composed).result().quasi_dists

print(f"Quasi distribution: {quasi_dists[0]}")

# saving results of a program
save_result({
    "quasi_dists": quasi_dists[0]
})

```

As you can see, we've imported our custom dependency `qiskit-experiments` and used its `StandardRB` module to generate an RB circuit, which we've composed with the circuit provided in the input arguments.

Let's create and configure out client

In [1]:
from quantum_serverless import QuantumServerless, GatewayProvider

In [2]:
provider = GatewayProvider(
    username="user",
    password="password123",
    host="http://localhost:8000",
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

In [3]:
from qiskit.circuit.random import random_circuit
circuit = random_circuit(2, 2)

To install custom dependency that our program might use we need to pass it as `dependencies` constructor argument for `Program` class. 
You can pass multiple dependencies and specify versions. 

In [4]:
from quantum_serverless import Program

program = Program(
    title="Program with dependencies",
    entrypoint="program_3.py",
    working_dir="./source_files/",
    dependencies=["qiskit-experiments"]
)

job = serverless.run_program(program, arguments={"circuit": circuit})
job

<Job | 7b175969-98fb-45b4-8d5f-1f9c3d4e63ab>

In [8]:
job.status()

'SUCCEEDED'

In [9]:
print(job.logs())

Quasi distribution: {0: 1.0}

